In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.power import TTestIndPower
from scipy.stats import pearsonr

# --------------------------
# Step 1: Simulation function
# --------------------------
def simulate_correlations(n_sessions, effect_size=0.0, n_trials=100):
    """
    Simulate correlation coefficients between LC spiking (Poisson) and
    pupil diameter (Gaussian) across n_sessions.

    Parameters:
        n_sessions (int): number of simulated experimental sessions
        effect_size (float): strength of correlation (0 = null)
        n_trials (int): number of trials per session
    Returns:
        corrs (array): correlation coefficients per session
    """
    corrs = []
    for _ in range(n_sessions):
        # Simulate LC spiking (Poisson distributed)
        lc = np.random.poisson(lam=5, size=n_trials)

        # Simulate pupil diameter (Gaussian distributed)
        pupil = np.random.normal(loc=0, scale=1, size=n_trials)

        # Add correlation effect if effect_size > 0
        pupil = pupil + effect_size * (lc - np.mean(lc))

        # Compute correlation
        r, _ = pearsonr(lc, pupil)
        corrs.append(r)
    return np.array(corrs)

# --------------------------
# Step 2: Power analysis setup
# --------------------------
analysis = TTestIndPower()

effect_sizes = np.linspace(0.1, 1.0, 10)  # possible effect sizes (Cohen's d)
sample_sizes = []

for es in effect_sizes:
    # For demonstration: compute required n to achieve 80% power
    n_needed = analysis.solve_power(effect_size=es, power=0.8, alpha=0.05, alternative='two-sided')
    sample_sizes.append(np.ceil(n_needed))

# --------------------------
# Step 3: Plot results
# --------------------------
plt.figure(figsize=(8,5))
plt.plot(effect_sizes, sample_sizes, marker='o')
plt.xlabel("Effect size (Cohen's d)")
plt.ylabel("Number of sessions needed (n)")
plt.title("Post-hoc power analysis (80% power)")
plt.grid(True)
plt.show()
